In [18]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import magic
import os
import nltk

# openai_api_key = os.getenv("OPENAI_API_KEY", "YourAPIKey")

# nltk.download('averaged_perceptron_tagger')

# pip install unstructured
# Other dependencies to install https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/unstructured_file.html
# pip install python-magic-bin
# pip install chromadb

import os
os.environ['OPENAI_API_KEY']=open('./../../keys.txt','r').read()
os.environ['SERPAPI_API_KEY']=open('./../../keys2.txt','r').read()
os.environ['GOOGLE_CSE_ID']=open('./../../keys3.txt','r').read()
os.environ['GOOGLE_API_KEY']=open('./../../keys4.txt','r').read()
os.environ['WOLFRAM_ALPHA_APPID'] = open('./../../keys5.txt','r').read()

In [2]:
# Get your loader ready
loader = DirectoryLoader('../data/PaulGrahamEssaySmall/', glob='**/*.txt')

In [3]:
# Load up your text into documents
documents = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [6]:
# Get your text splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [7]:
# Split your documents into texts
texts = text_splitter.split_documents(documents)

In [11]:
# Turn your texts into embeddings
embeddings = OpenAIEmbeddings()

In [22]:
# Get your docsearch ready
docsearch = FAISS.from_documents(texts, embeddings)
# docsearch = Chroma.from_documents(texts, embeddings)

In [16]:
# Load up your LLM
llm = OpenAI()

In [23]:
# Create your Retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
# qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch) # deprecated

In [24]:
# Run a query
query = "What did McCarthy discover?"
qa.run(query)

' McCarthy discovered a way to build a programming language using a handful of simple operators and a notation for functions. He called this language Lisp, for "List Processing," because one of his key ideas was to use a simple data structure called a list for both code and data.'

### Sources

In [25]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=docsearch.as_retriever(),
                                return_source_documents=True)
query = "What did McCarthy discover?"
result = qa({"query": query})

In [26]:
result['result']

' McCarthy discovered how to use a handful of simple operators and a notation for functions to build a whole programming language, which he called Lisp (for "List Processing"). He also discovered the idea of using a simple data structure called a list for both code and data.'

In [27]:
result['source_documents']

[Document(page_content='(I wrote this article to help myself understand exactly what McCarthy discovered. You don\'t need to know this stuff to program in Lisp, but it should be helpful to anyone who wants to understand the essence of Lisp \x97 both in the sense of its origins and its semantic core. The fact that it has such a core is one of Lisp\'s distinguishing features, and the reason why, unlike other languages, Lisp has dialects. )In 1960, John McCarthy published a remarkable paper in which he did for programming something like what Euclid did for geometry. He showed how, given a handful of simple operators and a notation for functions, you can build a whole programming language. He called this language Lisp, for "List Processing," because one of his key ideas was to use a simple data structure called a list for both code and data.It\'s worth understanding what McCarthy discovered, not just as a landmark in the history of computers, but as a model for what programming is tending 